# Part #2 Edge Detection

## 2(1) Apply the Gaussian filtering to the input image

In [1280]:
from functions import *

## 2(2) compute_image_gradient(img)

In [1281]:
def compute_image_gradient(img):
    sobel_x=np.array([[1,0,-1],[2,0,-2],[1,0,-1]],dtype=np.float64)
    sobel_y=np.array([[1,2,1],[0,0,0],[-1,-2,-1]],dtype=np.float64)
    start=time.time()
    d_x=cross_correlation_2d(img,sobel_x)
    d_y=cross_correlation_2d(img,sobel_y)
    magnitude=(d_x**2+d_y**2)**0.5
    np.clip(magnitude,0,255).astype(np.uint8)
    direction=np.arctan2(d_y,d_x)*180./np.pi%360
    print(np.min(direction),np.max(direction))
    # direction[direction<0]+=180
    # direction[direction>180]=360-direction[direction>180]
    # print(direction.shape)
    # # direction=np.arctan2(d_y,d_x)* 180./np.pi%360
    # img_show(direction[50:100,300:400])
    # direction=direction%360
    # print(direction[50][200:300].flatten())
    # direction[direction==90]=1
    # print(f"time: {time.time()-start}")
    return magnitude,direction

In [1282]:
import time,os

def get_gradient_img(img_path):
    img=get_gaussian_img(img_path,7,1.5)
    magnitude,direction=compute_image_gradient(img)
    # img_show(magnitude)
    directory="./result"
    if not os.path.exists(directory):
        os.makedirs(directory)
    cv2.imwrite(f'./result/part_2_edge_raw_{img_path}',magnitude) 
    return magnitude,direction

In [1283]:
LENNA_IMG_PATH="shapes.png"
magnitude,direction=get_gradient_img(LENNA_IMG_PATH)

0.0 359.4704382455157


## 2(3) NMS

In [1284]:
import numpy as np
def get_closet_angle(x,array=np.array([0,45,90,135,180,225,270,315,360])):
    return np.array([np.abs(x-angle) for angle in array]).argmin()%8


In [1285]:
# def non_max_suppression(G, theta):
#     #Height, Width
#     H,W = G.shape
#     #결과 저장 행렬 선언
#     res = np.zeros((H,W),dtype=np.int32)
#     #각 방향을 돌면서 계산
#     angle = theta * 180. / np.pi
#     angle[angle < 0] += 180

#     #(0,45,90,135) 방향을 돌면서 연산
#     for i in range(1,H-1):
#         for j in range(1,W-1):
#             # 0 degrees
#             if (0 <= angle[i, j] < 22.5) or (157.5 <= angle[i, j] <= 180):
#                 left = G[i, j - 1]
#                 right = G[i, j + 1]

#             # 45 degrees
#             elif 22.5 <= angle[i, j] < 67.5:
#                 left = G[i - 1, j + 1]
#                 right = G[i + 1, j - 1]

#             # 90 degrees
#             elif 67.5 <= angle[i, j] < 112.5:
#                 left = G[i - 1, j]
#                 right = G[i + 1, j]

#             # 135 degrees
#             elif 112.5 <= angle[i, j] < 157.5:
#                 left = G[i - 1, j - 1]
#                 right = G[i + 1, j + 1]

#             if (G[i, j] > left) and (G[i, j] > right):
#                 res[i, j] = G[i, j]
#     img_show(res)
#     cv2.imwrite(f'./result/part_2_nms_lenna.png',res)
#     return res

# non_max_suppression(magnitude,direction)

In [1286]:

# def non_maximum_suppression_dir(magnitude,direction):
#     direction_mapping = lambda x: get_closet_angle(x)
#     vec_direction_mapping = np.vectorize(direction_mapping) 
#     mappedIdx=vec_direction_mapping(direction)
#     # direction_idx=np.array([[0,1],[1,1],[1,0],[1,-1],[0,-1],[-1,-1],[-1,0],[-1,1]])
#     window_idx=np.array([5,2,1,0,3,6,7,8])
#     r,c=magnitude.shape
#     arr=np.zeros(magnitude.shape,dtype=np.float64)
#     for y in range(1,r-1):
#         for x in range(1,c-1):
#             tmp=magnitude[y-1:y+2,x-1:x+2].flatten()
#             angle=mappedIdx[y,x]
#             a_c=window_idx[angle]
#             b_c=window_idx[(angle+4)%8]
#             a_r=window_idx[(angle+2)%8]
#             b_r=window_idx[(angle+6)%8]
#             if tmp[a_r]<=tmp[4]>=tmp[b_r]: 
#                 if tmp[a_c]<=tmp[4]>=tmp[b_c]:
#                     arr[y,x]=tmp[4]
#             if tmp[a_c]<=tmp[4]>=tmp[b_c]:
#                 if tmp[a_r]<=tmp[4]>=tmp[b_r]: 
#                     arr[y,x]=tmp[4]
#             elif tmp[a_r]<tmp[4]>tmp[b_r]:   
#                 arr[y,x]=tmp[4]
#             elif tmp[a_c]<tmp[4]>tmp[b_c]:
#                 arr[y,x]=tmp[4]
                
#     cv2.imwrite(f'./result/part_2_nms_shapes.png',arr) 
    
#     # arr=np.zeros(magnitude.shape,dtype=np.int32)
#     # r,c=magnitude.shape
#     # for i in range(1,r-1):
#     #     for j in range(1,c-1):
#     #         if direction[i,j]==0 or direction[i,j]==180:
#     #             if magnitude[i,j-1]<magnitude[i,j]>magnitude[i,j+1]:
#     #                 arr[i,j]=magnitude[i,j]
#     #         elif direction[i,j]==45 or direction[i,j]==225:
#     #             if magnitude[i-1,j+1]<magnitude[i,j]>magnitude[i+1,j-1]:
#     #                 arr[i,j]=magnitude[i,j]
#     #         elif direction[i,j]==90 or direction[i,j]==270:
#     #             if magnitude[i-1,j]<magnitude[i,j]>magnitude[i+1,j]:
#     #                 arr[i,j]=magnitude[i,j]
#     #         elif direction[i,j]==135 and direction[i,j]==315:
#     #             if magnitude[i-1,j-1]<magnitude[i,j]>magnitude[i+1,j+1]:
#     #                 arr[i,j]=magnitude[i,j]
    
    
#     # for y in range(1,c-2):
#     #     for x in range(1,r-2):
#     #         if magnitude[y][x]==0:
#     #             arr[y,x]=0
#     #             continue
#     #         tmp=magnitude[y-1:y+2,x-1:x+2]
#     #         direction=mappedIdx[y,x]
#     #         ya,xa=window_idx[direction]
#     #         yb,xb=window_idx[(direction+4)%8]
#     #         if tmp[ya,xa]<=tmp[1,1]>=tmp[yb,xb]:
#     #             arr[y,x]=tmp[1,1]
#     #         else:
#     #             arr[y,x]=0
    
    
#     # arr = np.zeros(magnitude.shape)
#     # for i in range(1, c-1):
#     #     for j in range(1, r-1):
#     #         # 관심 영역 참조 통해 이웃 화소 가져오기
#     #         values = magnitude[i-1:i+2, j-1:j+2].flatten()
#     #         v1, v2 = values[mappedIdx[i, j]], values[(mappedIdx[i, j]+4)%8]
#     #         arr[i, j] = magnitude[i, j] if (v1 < magnitude[i, j] > v2) else 0
    
    
#     # arr = np.zeros((rows, cols), np.float32)
#     # for i in range(1, rows-1):
#     #     for j in range(1, cols-1):
#     #         # 관심 영역 참조 통해 이웃 화소 가져오기
#     #         tmp = magnitude[i-1:i+2, j-1:j+2].flatten()
#     #         first = [3, 0, 1, 2]
#     #         id = first[direction[i, j]]
#     #         v1, v2 = tmp[id], tmp[8-id]
#     #         arr[i, j] = magnitude[i, j] if (v1 < magnitude[i, j] > v2) else 0
                
#             # if np.max(tmp)>magnitude[y][x]:
#             #     magnitude[y][x]=0
#             # elif np.max(tmp)==magnitude[y][x]:
#             #     continue
#             # else:
#             #     arr[1][1]=magnitude[y][x]
#             #     magnitude[y-1:y+2,x-1:x+2]=arr
                
#     # for y in range(1,c-2):
#     #     for x in range(1,r-2):
#     #         tmp=magnitude[y-1:y+2,x-1:x+2]
#     #         will.append([y,x])
#     #         while will:
#     #             i,j=will.popleft()
#     #             ij=np.array([i,j])
#     #             idx=mappedIdx[i-1][j-1]
#     #             y1,x1=direction_idx[idx]+ij
#     #             y2,x2=direction_idx[(idx+4)%8]+ij
#     #             if magnitude[y1][x1]>magnitude[i][j] :
#     #                 magnitude[i][j]=0
#     #                 will.append([y1,x1])
#     #             if magnitude[y2][x2]>magnitude[i][j]:
#     #                 magnitude[i][j]=0
#     #                 will.append([y2,x2])

#     # windows=np.lib.stride_tricks.sliding_window_view(magnitude,(3,3))
#     # mag=[]
#     # for windowArr,idxArr in zip(windows,mappedIdx):
#     #     tmp=[]
#     #     for window, idx in zip(windowArr,idxArr):
#     #         maxi=window[1][1]
#     #         for value in [idx,(idx+4)%8]:
#     #             i,j=window_idx[value]
#     #             maxi=max(maxi,window[i][j])
#     #         tmp.append(maxi)
#     #     mag.append(tmp)
    
#     # for i,array in enumerate(mappedIdx):
#     #     for j,value in enumerate(array):
#     #         ij_array=np.array([i,j])
#     #         xa,ya=ij_array+direction_idx[value]
#     #         xb,yb=ij_array+direction_idx[(value+4)%8]
#             # if 0<=ya<c and 0<=xa<r and magnitude[ya][xa]<=magnitude[i][j]:
#             #     magnitude[ya][xa]=0
#             # elif 0<=yb<c and 0<=xb<r and magnitude[yb][xb]<=magnitude[i][j]:
#             #     magnitude[yb][xb]=0
#             # for idx in [value,(value+4)%8]:
#             #     y,x=direction_idx[idx]+ij_array
#             #     if 0<=x<r and 0<=y<c:
#             #         if magnitude[y][x]<magnitude[i][j]:
#             #       
#             # magnitude[y][x]=0
#     cv2.imwrite(f'./result/part_2_nms_shapes.png',arr) 
            
#     return arr
# non_maximum_suppression_dir(magnitude,direction)
       

In [1287]:
# img_show(cv2.Canny(cv2.imread("shapes.png", cv2.IMREAD_GRAYSCALE), 30, 70))



In [1288]:
def non_maximum_suppression_dir(magnitude,direction):
    direction_mapping = lambda x: get_closet_angle(x)
    vec_direction_mapping = np.vectorize(direction_mapping) 
    mappedIdx=vec_direction_mapping(direction)
    print(mappedIdx[50][200:300].flatten())
    # mappedIdx[mappedIdx==0]+=2
    # mappedIdx[mappedIdx==4]+=2
    # mappedIdx[mappedIdx==2]+=2
    # mappedIdx[mappedIdx==6]+=2
    direction_idx=np.array([[0,1],[1,1],[1,0],[1,-1],[0,-1],[-1,-1],[-1,0],[-1,1]])
    window_idx=np.array([5,2,1,0,3,6,7,8])
    r,c=magnitude.shape
    arr=np.zeros(magnitude.shape,dtype=np.float64)
    maxi=0
    for y in range(1,r-1):
        for x in range(1,c-1):
            if magnitude[y,x]!=0:
                tmp=magnitude[y-1:y+2,x-1:x+2].flatten()
                angle=mappedIdx[y,x]
                left=tmp[window_idx[angle]]
                right=tmp[window_idx[(angle+4)%8]]
                if left<=tmp[4]>=right:
                    arr[y,x]=tmp[4]
                if left==tmp[4]==right:
                    print(angle,magnitude[y,x])
            # elif left==tmp[4]>right:
            #     arr[y,x]=tmp[4]
            # elif left<tmp[4]==right:
            #     arr[y,x]=tmp[4]
                
                    
            # elif left==tmp[4]==right:
            #     print(mappedIdx[y,x])
            #     arr[y,x]=tmp[4]
    return arr

In [1289]:
suppressed_mag=non_maximum_suppression_dir(magnitude,direction)
cv2.imwrite(f'./result/part_2_nms_shapes.png',suppressed_mag) 

[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
1 0.1821912587640154
3 0.09109562938203784
1 0.030365209794072905
1 0.1821912587640154
1 1.955224401286697
3 1.1892197563952946
3 0.09109562938203784
1 0.030365209794072905
1 0.638063046742437
3 0.30175427232798274
1 0.1821912587640154
1 1.955224401286697
1 10.442989553845777
3 7.1722966469137575
3 1.1892197563952946
3 0.09109562938203784
1 0.030365209794072905
1 0.638063046742437
1 4.664883848915862
3 2.920751159852091
3 0.30175427232798274
1 0.1821912587640154
1 1.955224401286697
1 10.442989553845777
1 37.016451803378196
3 27.73217828029154
3 7.1722966469137575
3 1.1892197563952946
3 0.09109562938203784
1 0.030365209794072905
1 0.638063046742437
1 4.664883848915862
1 20.1980049262784
3 14.4139317413388
3 2.920751159852091
3 0.30175427232798274
1 0.1821912587640154
1 1.955224401286697

True

In [1290]:
def get_suppressed_mag(img_path):
    mag,direction=get_gradient_img(img_path)
    suppressed_mag=non_maximum_suppression_dir(mag,direction)
    cv2.imwrite(f'./result/part_2_edge_sup_{img_path}',suppressed_mag) 
